# Ch21: 自訂例外與 raise - 詳解範例本 Notebook 包含 **3 個完整的實務案例**,展示自訂例外在實際應用中的最佳實踐。## 📋 案例清單1. 表單驗證系統（自訂例外階層）2. 支付處理系統（PaymentError 階層）3. 遊戲狀態系統（GameError, InvalidMoveError）

---## 案例 1: 表單驗證系統### 情境建立一個使用者註冊系統,需要驗證:- Email 格式（包含 @、不為空）- 密碼強度（長度、複雜度）- 年齡範圍（18-120）- 使用者名稱（長度、字元限制）### 要處理的錯誤- `ValidationError`: 驗證錯誤的基底類別- `EmailError`: Email 格式錯誤- `PasswordError`: 密碼不符合規則- `AgeError`: 年齡不合法- `UsernameError`: 使用者名稱不合法### 學習重點- 建立例外階層- 為例外添加自訂屬性- 提供詳細的錯誤訊息

In [ ]:
# 第 1 步：定義例外階層class ValidationError(Exception):    """驗證錯誤的基底類別"""    passclass EmailError(ValidationError):    """Email 格式錯誤"""    def __init__(self, email, reason="格式不正確"):        self.email = email        self.reason = reason        super().__init__(f"無效的 Email '{email}'：{reason}")class PasswordError(ValidationError):    """密碼不符合規則"""    def __init__(self, reason):        self.reason = reason        super().__init__(f"密碼錯誤：{reason}")class AgeError(ValidationError):    """年齡不合法"""    def __init__(self, age, min_age=18, max_age=120):        self.age = age        self.min_age = min_age        self.max_age = max_age        super().__init__(            f"年齡 {age} 不在有效範圍內（{min_age}-{max_age} 歲）"        )class UsernameError(ValidationError):    """使用者名稱不合法"""    def __init__(self, username, reason):        self.username = username        self.reason = reason        super().__init__(f"無效的使用者名稱 '{username}'：{reason}")print("✅ 例外階層定義完成")print("\n例外繼承關係：")print("ValidationError")print("├── EmailError")print("├── PasswordError")print("├── AgeError")print("└── UsernameError")

In [ ]:
# 第 2 步：實作驗證函式def validate_email(email):    """    驗證 Email 格式    Raises:        EmailError: Email 格式不正確時    """    if not email:        raise EmailError(email, "Email 不能為空")    if '@' not in email:        raise EmailError(email, "Email 必須包含 @")    if email.count('@') > 1:        raise EmailError(email, "Email 只能包含一個 @")    local, domain = email.split('@')    if not local:        raise EmailError(email, "@ 前面不能為空")    if not domain:        raise EmailError(email, "@ 後面不能為空")    if '.' not in domain:        raise EmailError(email, "網域必須包含 .")    print(f"✅ Email 驗證通過：{email}")    return Truedef validate_password(password):    """    驗證密碼強度    規則：    - 長度至少 8 字元    - 包含大寫字母    - 包含小寫字母    - 包含數字    Raises:        PasswordError: 密碼不符合規則時    """    if len(password) < 8:        raise PasswordError("密碼長度必須至少 8 字元")    if not any(c.isupper() for c in password):        raise PasswordError("密碼必須包含至少一個大寫字母")    if not any(c.islower() for c in password):        raise PasswordError("密碼必須包含至少一個小寫字母")    if not any(c.isdigit() for c in password):        raise PasswordError("密碼必須包含至少一個數字")    print(f"✅ 密碼驗證通過（長度：{len(password)}）")    return Truedef validate_age(age):    """    驗證年齡    Raises:        AgeError: 年齡不在 18-120 之間時    """    if age < 18:        raise AgeError(age)    if age > 120:        raise AgeError(age)    print(f"✅ 年齡驗證通過：{age} 歲")    return Truedef validate_username(username):    """    驗證使用者名稱    規則：    - 長度 3-20 字元    - 只能包含字母、數字、底線    - 必須以字母開頭    Raises:        UsernameError: 使用者名稱不合法時    """    if len(username) < 3:        raise UsernameError(username, "長度必須至少 3 字元")    if len(username) > 20:        raise UsernameError(username, "長度不能超過 20 字元")    if not username[0].isalpha():        raise UsernameError(username, "必須以字母開頭")    if not all(c.isalnum() or c == '_' for c in username):        raise UsernameError(username, "只能包含字母、數字、底線")    print(f"✅ 使用者名稱驗證通過：{username}")    return Trueprint("✅ 驗證函式定義完成")

In [ ]:
# 第 3 步：整合驗證系統class UserRegistration:    """使用者註冊系統"""    def __init__(self):        self.users = []  # 儲存已註冊的使用者    def register(self, username, email, password, age):        """        註冊新使用者        Returns:            dict: 使用者資料（成功時）        Raises:            ValidationError: 任何驗證失敗時        """        print(f"\n=== 開始註冊使用者：{username} ===")        # 依序驗證所有欄位        validate_username(username)        validate_email(email)        validate_password(password)        validate_age(age)        # 所有驗證通過，建立使用者        user = {            "username": username,            "email": email,            "age": age        }        self.users.append(user)        print(f"\n🎉 註冊成功！歡迎 {username}")        return user# 測試系統registration = UserRegistration()print("=== 測試 1：正確的資料 ===")try:    user = registration.register(        username="alice_123",        email="alice@example.com",        password="SecurePass123",        age=25    )    print(f"使用者資料：{user}")except ValidationError as e:    print(f"❌ 註冊失敗：{e}")

In [ ]:
# 測試各種錯誤情況print("\n=== 測試 2：Email 錯誤 ===")try:    registration.register("bob", "invalid-email", "Password123", 30)except EmailError as e:    print(f"❌ {type(e).__name__}: {e}")    print(f"   錯誤的 Email: {e.email}")    print(f"   錯誤原因: {e.reason}")print("\n=== 測試 3：密碼錯誤 ===")try:    registration.register("charlie", "charlie@example.com", "weak", 28)except PasswordError as e:    print(f"❌ {type(e).__name__}: {e}")    print(f"   錯誤原因: {e.reason}")print("\n=== 測試 4：年齡錯誤 ===")try:    registration.register("david", "david@example.com", "GoodPass123", 15)except AgeError as e:    print(f"❌ {type(e).__name__}: {e}")    print(f"   輸入年齡: {e.age}")    print(f"   有效範圍: {e.min_age}-{e.max_age}")print("\n=== 測試 5：使用者名稱錯誤 ===")try:    registration.register("ab", "eve@example.com", "GoodPass123", 22)except UsernameError as e:    print(f"❌ {type(e).__name__}: {e}")    print(f"   錯誤的使用者名稱: {e.username}")    print(f"   錯誤原因: {e.reason}")

In [ ]:
# 第 4 步：分層捕獲例外def safe_register(registration, username, email, password, age):    """    安全的註冊函式，展示分層捕獲    """    try:        return registration.register(username, email, password, age)    except EmailError as e:        # 特定處理 Email 錯誤        print(f"\n📧 Email 驗證失敗")        print(f"   建議：請檢查 Email 格式是否正確")        return None    except PasswordError as e:        # 特定處理密碼錯誤        print(f"\n🔒 密碼強度不足")        print(f"   建議：密碼需包含大小寫字母和數字，長度至少 8 字元")        return None    except ValidationError as e:        # 處理其他所有驗證錯誤        print(f"\n⚠️  驗證失敗：{e}")        return None# 測試分層捕獲print("=== 測試分層捕獲 ===")safe_register(registration, "test", "bad@email", "Weak1", 20)safe_register(registration, "test", "test@example.com", "weak", 20)safe_register(registration, "test", "test@example.com", "GoodPass123", 150)print("\n✅ 案例 1 完成：學會設計自訂例外階層與驗證系統")

---## 案例 2: 支付處理系統### 情境建立一個電子支付系統,需要處理:- 餘額不足- 信用卡過期- 交易限額超過- 支付閘道錯誤（使用例外鏈）### 要處理的錯誤- `PaymentError`: 支付錯誤的基底類別- `InsufficientFundsError`: 餘額不足- `CardExpiredError`: 信用卡過期- `TransactionLimitError`: 超過交易限額- `PaymentGatewayError`: 支付閘道錯誤（包裝原始錯誤）### 學習重點- 例外階層設計- 例外鏈（raise...from）- 攜帶豐富的除錯資訊

In [ ]:
# 第 1 步：定義支付例外階層from datetime import datetimeclass PaymentError(Exception):    """支付錯誤的基底類別"""    passclass InsufficientFundsError(PaymentError):    """餘額不足"""    def __init__(self, account_id, balance, amount):        self.account_id = account_id        self.balance = balance        self.amount = amount        self.shortage = amount - balance        super().__init__(            f"帳戶 {account_id} 餘額不足："            f"需要 ${amount}，但只有 ${balance}（不足 ${self.shortage}）"        )class CardExpiredError(PaymentError):    """信用卡過期"""    def __init__(self, card_number, expiry_date):        self.card_number = card_number[-4:]  # 只顯示後 4 碼        self.expiry_date = expiry_date        super().__init__(            f"信用卡 ****{self.card_number} 已過期（到期日：{expiry_date}）"        )class TransactionLimitError(PaymentError):    """超過交易限額"""    def __init__(self, amount, daily_limit, used_today):        self.amount = amount        self.daily_limit = daily_limit        self.used_today = used_today        self.remaining = daily_limit - used_today        super().__init__(            f"超過每日交易限額："            f"本次交易 ${amount}，今日已用 ${used_today}/${daily_limit}，"            f"剩餘額度 ${self.remaining}"        )class PaymentGatewayError(PaymentError):    """支付閘道錯誤（包裝原始錯誤）"""    def __init__(self, gateway_name, transaction_id, original_error):        self.gateway_name = gateway_name        self.transaction_id = transaction_id        self.original_error = original_error        super().__init__(            f"支付閘道 {gateway_name} 發生錯誤（交易 ID: {transaction_id}）"        )print("✅ 支付例外階層定義完成")print("\nPaymentError")print("├── InsufficientFundsError")print("├── CardExpiredError")print("├── TransactionLimitError")print("└── PaymentGatewayError")

In [ ]:
# 第 2 步：模擬支付閘道import randomclass MockPaymentGateway:    """模擬的支付閘道（會隨機失敗）"""    def __init__(self, name, failure_rate=0.3):        self.name = name        self.failure_rate = failure_rate    def process(self, amount, card_number):        """處理支付（可能失敗）"""        # 隨機產生錯誤        if random.random() < self.failure_rate:            error_type = random.choice(['timeout', 'network', 'declined'])            if error_type == 'timeout':                raise TimeoutError("支付閘道逾時")            elif error_type == 'network':                raise ConnectionError("網路連線失敗")            else:                raise ValueError("銀行拒絕交易")        # 成功        transaction_id = f"TXN{random.randint(100000, 999999)}"        return transaction_idprint("✅ 模擬支付閘道定義完成")

In [ ]:
# 第 3 步：實作支付系統class PaymentProcessor:    """支付處理器"""    def __init__(self, gateway_name="StripeGateway"):        self.gateway = MockPaymentGateway(gateway_name)        self.daily_limit = 50000  # 每日限額 $50,000        self.used_today = 0  # 今日已使用額度    def charge(self, account_id, balance, amount, card_number, expiry_date):        """        執行支付        Returns:            str: 交易 ID        Raises:            PaymentError: 各種支付錯誤        """        print(f"\n=== 處理支付：${amount} ===")        # 驗證 1：檢查餘額        if amount > balance:            raise InsufficientFundsError(account_id, balance, amount)        # 驗證 2：檢查卡片是否過期        expiry = datetime.strptime(expiry_date, "%Y-%m")        if expiry < datetime.now():            raise CardExpiredError(card_number, expiry_date)        # 驗證 3：檢查交易限額        if self.used_today + amount > self.daily_limit:            raise TransactionLimitError(amount, self.daily_limit, self.used_today)        # 呼叫支付閘道        try:            transaction_id = self.gateway.process(amount, card_number)        except (TimeoutError, ConnectionError, ValueError) as e:            # 使用例外鏈包裝原始錯誤            raise PaymentGatewayError(                self.gateway.name,                "PENDING",                str(e)            ) from e        # 成功        self.used_today += amount        print(f"✅ 支付成功！交易 ID: {transaction_id}")        print(f"   今日已用額度: ${self.used_today}/{self.daily_limit}")        return transaction_idprint("✅ 支付處理器定義完成")

In [ ]:
# 第 4 步：測試各種支付情況processor = PaymentProcessor()# 測試 1：正常支付print("=== 測試 1：正常支付 ===")try:    txn_id = processor.charge(        account_id="ACC001",        balance=10000,        amount=5000,        card_number="1234567890123456",        expiry_date="2026-12"    )except PaymentError as e:    print(f"❌ 支付失敗：{e}")

In [ ]:
# 測試 2：餘額不足print("\n=== 測試 2：餘額不足 ===")try:    processor.charge(        account_id="ACC002",        balance=1000,        amount=5000,        card_number="1234567890123456",        expiry_date="2026-12"    )except InsufficientFundsError as e:    print(f"❌ {type(e).__name__}")    print(f"   帳戶: {e.account_id}")    print(f"   餘額: ${e.balance}")    print(f"   需要: ${e.amount}")    print(f"   不足: ${e.shortage}")

In [ ]:
# 測試 3：信用卡過期print("\n=== 測試 3：信用卡過期 ===")try:    processor.charge(        account_id="ACC003",        balance=10000,        amount=3000,        card_number="1234567890123456",        expiry_date="2020-01"  # 已過期    )except CardExpiredError as e:    print(f"❌ {type(e).__name__}")    print(f"   卡號後四碼: {e.card_number}")    print(f"   到期日: {e.expiry_date}")

In [ ]:
# 測試 4：超過交易限額print("\n=== 測試 4：超過交易限額 ===")processor.used_today = 48000  # 模擬今日已用 $48,000try:    processor.charge(        account_id="ACC004",        balance=100000,        amount=5000,  # 48000 + 5000 > 50000        card_number="1234567890123456",        expiry_date="2026-12"    )except TransactionLimitError as e:    print(f"❌ {type(e).__name__}")    print(f"   本次交易: ${e.amount}")    print(f"   今日已用: ${e.used_today}")    print(f"   每日限額: ${e.daily_limit}")    print(f"   剩餘額度: ${e.remaining}")

In [ ]:
# 測試 5：支付閘道錯誤（例外鏈）print("\n=== 測試 5：支付閘道錯誤（會隨機發生）===")processor.used_today = 0  # 重置# 多次嘗試，直到觸發閘道錯誤for i in range(5):    try:        print(f"\n第 {i+1} 次嘗試...")        txn_id = processor.charge(            account_id="ACC005",            balance=10000,            amount=1000,            card_number="1234567890123456",            expiry_date="2026-12"        )    except PaymentGatewayError as e:        print(f"\n❌ {type(e).__name__}")        print(f"   閘道: {e.gateway_name}")        print(f"   原始錯誤: {e.original_error}")        print(f"   例外鏈 (__cause__): {e.__cause__}")        break    except PaymentError as e:        print(f"❌ 其他支付錯誤：{e}")print("\n✅ 案例 2 完成：學會設計支付系統的例外階層與例外鏈")

---## 案例 3: 遊戲狀態系統### 情境建立一個回合制遊戲,需要處理:- 無效的移動（棋子移動規則）- 遊戲狀態錯誤（未輪到玩家、遊戲已結束）- 棋盤邊界檢查- 重新拋出例外（re-raise）### 要處理的錯誤- `GameError`: 遊戲錯誤的基底類別- `InvalidMoveError`: 無效的移動- `GameStateError`: 遊戲狀態錯誤- `OutOfBoundsError`: 超出棋盤邊界### 學習重點- 狀態驗證與例外- 重新拋出例外（raise）- assert vs raise 的選擇

In [ ]:
# 第 1 步：定義遊戲例外class GameError(Exception):    """遊戲錯誤的基底類別"""    passclass InvalidMoveError(GameError):    """無效的移動"""    def __init__(self, piece, from_pos, to_pos, reason):        self.piece = piece        self.from_pos = from_pos        self.to_pos = to_pos        self.reason = reason        super().__init__(            f"無效的移動：{piece} 從 {from_pos} 到 {to_pos}（{reason}）"        )class GameStateError(GameError):    """遊戲狀態錯誤"""    def __init__(self, current_state, attempted_action):        self.current_state = current_state        self.attempted_action = attempted_action        super().__init__(            f"無效的操作：遊戲狀態為 '{current_state}'，無法執行 '{attempted_action}'"        )class OutOfBoundsError(GameError):    """超出棋盤邊界"""    def __init__(self, position, board_size):        self.position = position        self.board_size = board_size        super().__init__(            f"位置 {position} 超出棋盤範圍（棋盤大小：{board_size}x{board_size}）"        )print("✅ 遊戲例外定義完成")print("\nGameError")print("├── InvalidMoveError")print("├── GameStateError")print("└── OutOfBoundsError")

In [ ]:
# 第 2 步：實作簡單的西洋棋遊戲class ChessGame:    """簡化的西洋棋遊戲（只實作基本移動）"""    def __init__(self, board_size=8):        self.board_size = board_size        self.current_player = "White"  # White 或 Black        self.turn_count = 0        self.game_over = False        self.winner = None        self.board = {}  # {(row, col): piece_name}        # 初始化棋子        self.board[(0, 0)] = "White Rook"        self.board[(7, 7)] = "Black Rook"    def _is_valid_position(self, pos):        """檢查位置是否在棋盤內"""        row, col = pos        return 0 <= row < self.board_size and 0 <= col < self.board_size    def _validate_game_state(self, action):        """驗證遊戲狀態"""        if self.game_over:            state = f"已結束（勝者：{self.winner}）"            raise GameStateError(state, action)    def move(self, piece_owner, from_pos, to_pos):        """        移動棋子        Args:            piece_owner: 棋子擁有者（'White' 或 'Black'）            from_pos: 起始位置 (row, col)            to_pos: 目標位置 (row, col)        Raises:            GameError: 各種遊戲錯誤        """        print(f"\n=== 回合 {self.turn_count + 1}：{piece_owner} 的回合 ===")        try:            # 驗證 1：遊戲狀態            self._validate_game_state("移動棋子")            # 驗證 2：是否輪到該玩家            if piece_owner != self.current_player:                raise GameStateError(                    f"{self.current_player} 的回合",                    f"{piece_owner} 嘗試移動"                )            # 驗證 3：起始位置是否在棋盤內            if not self._is_valid_position(from_pos):                raise OutOfBoundsError(from_pos, self.board_size)            # 驗證 4：目標位置是否在棋盤內            if not self._is_valid_position(to_pos):                raise OutOfBoundsError(to_pos, self.board_size)            # 驗證 5：起始位置是否有棋子            if from_pos not in self.board:                raise InvalidMoveError(                    "無",                    from_pos,                    to_pos,                    "起始位置沒有棋子"                )            piece = self.board[from_pos]            # 驗證 6：棋子是否屬於當前玩家            if not piece.startswith(piece_owner):                raise InvalidMoveError(                    piece,                    from_pos,                    to_pos,                    f"這個棋子不屬於 {piece_owner}"                )            # 驗證 7：城堡（Rook）的移動規則（只能直線移動）            if "Rook" in piece:                from_row, from_col = from_pos                to_row, to_col = to_pos                # 必須是直線移動（同行或同列）                if from_row != to_row and from_col != to_col:                    raise InvalidMoveError(                        piece,                        from_pos,                        to_pos,                        "城堡只能直線移動（同行或同列）"                    )            # 執行移動            del self.board[from_pos]            self.board[to_pos] = piece            # 切換玩家            self.current_player = "Black" if self.current_player == "White" else "White"            self.turn_count += 1            print(f"✅ 移動成功：{piece} 從 {from_pos} 到 {to_pos}")            return True        except GameError:            # 重新拋出遊戲相關的例外            print(f"❌ 移動失敗")            raise  # re-raise：保留原始 traceback    def end_game(self, winner):        """結束遊戲"""        self.game_over = True        self.winner = winner        print(f"\n🏆 遊戲結束！勝者：{winner}")print("✅ ChessGame 類別定義完成")

In [ ]:
# 第 3 步：測試遊戲game = ChessGame()# 測試 1：正常移動print("=== 測試 1：正常移動 ===")try:    game.move("White", (0, 0), (0, 5))  # 白色城堡向右移動except GameError as e:    print(f"錯誤：{e}")

In [ ]:
# 測試 2：不是自己的回合print("\n=== 測試 2：不是自己的回合 ===")try:    game.move("White", (0, 5), (0, 6))  # White 連續移動兩次except GameStateError as e:    print(f"❌ {type(e).__name__}")    print(f"   當前狀態: {e.current_state}")    print(f"   嘗試的操作: {e.attempted_action}")

In [ ]:
# 測試 3：超出棋盤邊界print("\n=== 測試 3：超出棋盤邊界 ===")try:    game.move("Black", (7, 7), (10, 10))  # 超出 8x8 棋盤except OutOfBoundsError as e:    print(f"❌ {type(e).__name__}")    print(f"   位置: {e.position}")    print(f"   棋盤大小: {e.board_size}x{e.board_size}")

In [ ]:
# 測試 4：違反移動規則print("\n=== 測試 4：違反移動規則 ===")try:    game.move("Black", (7, 7), (5, 5))  # 城堡不能斜著走except InvalidMoveError as e:    print(f"❌ {type(e).__name__}")    print(f"   棋子: {e.piece}")    print(f"   起始: {e.from_pos}")    print(f"   目標: {e.to_pos}")    print(f"   原因: {e.reason}")

In [ ]:
# 測試 5：正常移動（Black）print("\n=== 測試 5：Black 的回合 ===")try:    game.move("Black", (7, 7), (7, 3))  # 黑色城堡向左移動except GameError as e:    print(f"錯誤：{e}")

In [ ]:
# 測試 6：遊戲結束後嘗試移動print("\n=== 測試 6：遊戲結束後嘗試移動 ===")game.end_game("White")try:    game.move("White", (0, 5), (0, 6))except GameStateError as e:    print(f"❌ {type(e).__name__}")    print(f"   當前狀態: {e.current_state}")    print(f"   嘗試的操作: {e.attempted_action}")print("\n✅ 案例 3 完成：學會在遊戲系統中使用自訂例外管理狀態")

---## 🎉 三個案例完成！### 總結您已完成 3 個實務案例：1. **表單驗證系統** ✅   - 建立 ValidationError 階層   - 為每種驗證錯誤設計專屬例外   - 攜帶詳細的錯誤資訊（email, reason, age, etc.）   - 分層捕獲例外2. **支付處理系統** ✅   - 建立 PaymentError 階層   - 使用例外鏈（raise...from）包裝閘道錯誤   - 攜帶豐富的交易資訊（帳戶、金額、限額）   - 處理各種支付情境3. **遊戲狀態系統** ✅   - 建立 GameError 階層   - 使用 raise 重新拋出例外   - 驗證遊戲狀態與規則   - 區分內部檢查（assert）與外部驗證（raise）### 核心技能- ✅ 設計清晰的例外階層（3 層結構）- ✅ 為例外添加自訂屬性（攜帶除錯資訊）- ✅ 使用 raise...from 建立例外鏈- ✅ 重新拋出例外（raise）保留 traceback- ✅ 分層捕獲例外（specific → general）- ✅ 撰寫清晰的錯誤訊息### 設計模式總結**例外階層的三層結構**：```BaseError (基底類別)├── CategoryError1 (分類)│   ├── SpecificError1A (具體錯誤)│   └── SpecificError1B└── CategoryError2    └── SpecificError2A```**自訂屬性的模板**：```pythonclass MyError(BaseError):    def __init__(self, arg1, arg2):        self.arg1 = arg1  # 儲存屬性        self.arg2 = arg2        message = f"錯誤：{arg1}, {arg2}"  # 建立訊息        super().__init__(message)  # 呼叫父類```### 下一步- 完成 **03-practice.ipynb** (6 題課堂練習)- 完成 **04-exercises.ipynb** (10 題課後習題)- 挑戰 **quiz.ipynb** (自我測驗)